In [489]:
from tabulate import tabulate
import pandas as pd
import importlib
import util.Data as data_util
import util.DataframeProcess as df_proc
import json

In [490]:
importlib.reload(data_util)
importlib.reload(df_proc)

<module 'util.DataframeProcess' from 'D:\\scul\\uaic-ub\\licenta\\code\\util\\DataframeProcess.py'>

In [491]:
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_colwidth', 40)

# Data

In [492]:
csv_filename = r'../data_db/final_coordinates-conversion-v2.csv'
df_main = data_util.import_data(csv_filename)

In [493]:
df_data = df_main.copy()
df_data.shape

(1827, 26)

## all reports

## Focis number

In [494]:
#V1
df_data['counter'] = 1
ds_focis = df_data.groupby(['title', 'author'])['counter'].sum()

In [495]:
#V2
print("Count of Each group:")
df_group_focis = df_data.groupby(['title', 'author']
                                 ).size().reset_index(name="#Foci")
df_group_focis

Count of Each group:


,title,author,#Foci
0,A Rapid fMRI Task Battery for Mappin...,Drobyshevsky et al,13
1,A functional neuroimaging study of m...,Taylor et al,41
2,A region of mesial prefrontal cortex...,Knutson et al,21
3,Abnormal neural activity in partiall...,Lee et al,58
4,Activity in human reward-sensitive b...,Nieuwenhuis et al,10
...,...,...,...
56,Stimulus-dependent modulation of wor...,Duggirala et al,62
57,The Architecture of Reward Value Cod...,Sescousse et al,46
58,The neural system that bridges rewar...,Pochon et al,43
59,Tracking the Hemodynamic Responses t...,Delgado et al,32


## Subjects number

In [496]:
# group_subjects = df_data.groupby(['title','author']).agg({'subjects':['mean','min','max']})
# group_subjects = df_data.groupby(['title', 'author', 'subjects'])
# for key, item in group_subjects:
#     print(type(key), "\n\n")
#     break

# group_subjects.describe()

In [497]:
group_subjects = df_data.groupby(['title', 'author', 'subjects'])
dict_keys_subjects = group_subjects.groups.keys()
df_group_subjects = pd.DataFrame.from_records([*dict_keys_subjects], columns=['title', 'author', '#Subjects'])
df_group_subjects.head()

,title,author,#Subjects
0,A Rapid fMRI Task Battery for Mappin...,Drobyshevsky et al,31
1,A functional neuroimaging study of m...,Taylor et al,12
2,A region of mesial prefrontal cortex...,Knutson et al,12
3,Abnormal neural activity in partiall...,Lee et al,14
4,Activity in human reward-sensitive b...,Nieuwenhuis et al,14


In [498]:
df_final = df_group_focis.join(df_group_subjects[['title', '#Subjects']].set_index('title'), on='title')
df_final.head()

,title,author,#Foci,#Subjects
0,A Rapid fMRI Task Battery for Mappin...,Drobyshevsky et al,13,31
1,A functional neuroimaging study of m...,Taylor et al,41,12
2,A region of mesial prefrontal cortex...,Knutson et al,21,12
3,Abnormal neural activity in partiall...,Lee et al,58,14
4,Activity in human reward-sensitive b...,Nieuwenhuis et al,10,14


## Get set of contrast

In [499]:
# ds_group_contrast = df_data.groupby(['title', 'author'])['contrast'].apply(set)
# df_group_contrast = ds_group_contrast.to_frame().reset_index()
# df_group_contrast
# # df[['contrast']].iloc[0][0] <- set

In [500]:
# df_group_contrast = df_data.groupby(['title', 'author'])['contrast'].apply(set).reset_index(name='contrast_set')
df_group_contrast = df_data.groupby(['title', 'author'])['contrast'].apply(lambda x: set(x)).reset_index(
    name='contrast_set')
df_group_contrast.head(5)

,title,author,contrast_set
0,A Rapid fMRI Task Battery for Mappin...,Drobyshevsky et al,{2-back > 0-back}
1,A functional neuroimaging study of m...,Taylor et al,{-}
2,A region of mesial prefrontal cortex...,Knutson et al,"{lose > no reward, reward > no reward}"
3,Abnormal neural activity in partiall...,Lee et al,{1-back > rest}
4,Activity in human reward-sensitive b...,Nieuwenhuis et al,{high reward > low reward}


In [501]:
df_group_contrast[['contrast_set']].iloc[2][0]

{'lose > no reward', 'reward > no reward'}

In [502]:
df_final = df_final.join(df_group_contrast[['title', 'contrast_set']].set_index('title'), on='title')
df_final.head()

,title,author,#Foci,#Subjects,contrast_set
0,A Rapid fMRI Task Battery for Mappin...,Drobyshevsky et al,13,31,{2-back > 0-back}
1,A functional neuroimaging study of m...,Taylor et al,41,12,{-}
2,A region of mesial prefrontal cortex...,Knutson et al,21,12,"{lose > no reward, reward > no reward}"
3,Abnormal neural activity in partiall...,Lee et al,58,14,{1-back > rest}
4,Activity in human reward-sensitive b...,Nieuwenhuis et al,10,14,{high reward > low reward}


## Get set of keywords

In [503]:
#V1
# df_group_keywords = df_data.groupby(['title', 'author'])['keywords'].apply(set).reset_index(name='keywords_set')
df_group_keywords = df_data.groupby(['title', 'author'])['keywords'].apply(lambda x: set(x)).reset_index(
    name='keywords_set')
# ds_group_keywords = df_test.groupby(['title', 'author']).agg({"keywords": lambda x: set(x)})#%%

#df_group_keywords[['keywords_set']].iloc[0][0] <- set
df_group_keywords.head()

,title,author,keywords_set
0,A Rapid fMRI Task Battery for Mappin...,Drobyshevsky et al,{fMRI; working memory; talairach; n-...
1,A functional neuroimaging study of m...,Taylor et al,{fMRI; reward value; monetary reward...
2,A region of mesial prefrontal cortex...,Knutson et al,{fMRI; reward value; anticipation ph...
3,Abnormal neural activity in partiall...,Lee et al,{fMRI; working memory; talairach; n-...
4,Activity in human reward-sensitive b...,Nieuwenhuis et al,{fMRI; reward magnitude; reward valu...


In [504]:
df_group_keywords['keywords_split'] = df_group_keywords['keywords_set'].apply(df_proc.split_keys)
df_group_keywords.head()

,title,author,keywords_set,keywords_split
0,A Rapid fMRI Task Battery for Mappin...,Drobyshevsky et al,{fMRI; working memory; talairach; n-...,"{fMRI, talairach, n-back task, worki..."
1,A functional neuroimaging study of m...,Taylor et al,{fMRI; reward value; monetary reward...,"{fMRI, monetary reward, reward value..."
2,A region of mesial prefrontal cortex...,Knutson et al,{fMRI; reward value; anticipation ph...,"{fMRI, talairach, monetary reward, r..."
3,Abnormal neural activity in partiall...,Lee et al,{fMRI; working memory; talairach; n-...,"{fMRI, talairach, n-back task, worki..."
4,Activity in human reward-sensitive b...,Nieuwenhuis et al,{fMRI; reward magnitude; reward valu...,"{fMRI, talairach, context-dependent ..."


In [505]:
df_group_keywords[['keywords_split']].iloc[0][0]

{'fMRI',
 'n-back task',
 'talairach',
 'two-back task',
 'visual letters',
 'working memory'}

In [506]:
df_final = df_final.join(df_group_keywords[['title', 'keywords_split']].set_index('title'), on='title')
df_final.head(3)

,title,author,#Foci,#Subjects,contrast_set,keywords_split
0,A Rapid fMRI Task Battery for Mappin...,Drobyshevsky et al,13,31,{2-back > 0-back},"{fMRI, talairach, n-back task, worki..."
1,A functional neuroimaging study of m...,Taylor et al,41,12,{-},"{fMRI, monetary reward, reward value..."
2,A region of mesial prefrontal cortex...,Knutson et al,21,12,"{lose > no reward, reward > no reward}","{fMRI, talairach, monetary reward, r..."


In [507]:
df_final.shape

(61, 6)

## Print latex to text file

In [508]:
split_no = 35
df_final_print = df_final.drop(['title'], axis=1)
df_final_print = df_final_print.sort_values(by=['author'])
df_final_print = [df_final_print.iloc[:split_no, :], df_final_print.iloc[split_no:, :]]

In [509]:
# pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 40)
df_final_print[0].head()

,author,#Foci,#Subjects,contrast_set,keywords_split
33,Abler et al,43,15,"{expected reward > received reward, ...","{fMRI, anticipation phase, anova, in..."
12,Aharon et al,51,6,"{all female faces > all male faces, ...","{passive, fMRI, talairach, social re..."
15,Akitsuki et al,35,36,"{win after losing streak > w1, all w...","{fMRI, talairach, monetary reward, d..."
6,Beneventi et al,24,12,{2-back > 1-back > 0-back},"{fMRI, n-back task, working memory, ..."
34,Carrion et al,34,13,"{3-back > 0-back, 2-back > 0-back}","{fMRI, two-back task, MNI, three-bac..."


In [510]:
columns_to_print = ['author', '#Foci', '#Subjects']
pd.set_option('display.max_colwidth', None)  # to show all the column values (-1)
latex_filepath = r'../data_db/latex/all_reports.txt'
with open(latex_filepath, 'w') as f:
    for df in df_final_print:
        f.write(df[columns_to_print].to_latex(index=False))
pd.set_option('display.max_colwidth', 40)

C:\Users\Chh\AppData\Local\Temp\ipykernel_31044\1855274459.py:6: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(df[columns_to_print].to_latex(index=False))
C:\Users\Chh\AppData\Local\Temp\ipykernel_31044\1855274459.py:6: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(df[columns_to_print].to_latex(index=False))


## Checks

### Contrast

In [511]:
list_of_contrast_set = list(df_final['contrast_set'])

In [512]:
set_all_contrasts = set()
for s in list_of_contrast_set:
    set_all_contrasts.update(s)
len(set_all_contrasts)

79

In [513]:
counter_contrast = data_util.df_counter(df_final.explode('contrast_set'), 'contrast_set',
                                        set_keywords=set_all_contrasts, order=1)
counter_contrast

{'>nan_values<': 0,
 'lose > no reward': 1,
 '1-back > 2-back': 1,
 'cash-out > control': 1,
 'response maintenance > response switch': 1,
 'inhibiton task > control task': 1,
 'response switch > response maintenance': 1,
 'erotic reward > monetary reward': 1,
 'win after losing streak > w1': 1,
 'retrocue > precue': 1,
 'social reward magnitude': 1,
 'ar switch > stay and sr switch > stay': 1,
 'low-calorie > high-calorie': 1,
 'beautiful faces > average faces': 1,
 'decision-making > control task': 1,
 'negative correlation with reward level': 1,
 'low-calorie and high-calorie': 1,
 'movement > no movement and reward > no reward': 1,
 'anticipation phase': 1,
 'w1 > l1': 1,
 'monetary reward > faces reward': 1,
 'beautiful male > average male': 1,
 'ar; sr > neutral activity': 1,
 'sub-dif > frame-dif': 1,
 'reward during run of success': 1,
 'monetary reward magnitude': 1,
 'average female > average male': 1,
 'win modulation': 1,
 'disadvantageous > advantageous': 1,
 'expected rew

### keywords

In [514]:
list_of_keywords_set = list(df_final['keywords_split'])

In [515]:
set_all_keywords = set()
for s in list_of_keywords_set:
    set_all_keywords.update(s)
len(set_all_keywords)

96

In [516]:
counter_keywords = data_util.df_counter(df_final.explode('keywords_split'), 'keywords_split',
                                        set_keywords=set_all_keywords, order=1)
counter_keywords

{'>nan_values<': 0,
 'verbal pseudo-words': 1,
 'potential loss': 1,
 'response maintenance': 1,
 'anova': 1,
 'increasing reward magnitude': 1,
 'increasing reward uncertainty': 1,
 'effect of reward': 1,
 'chocolate bars': 1,
 'time effect': 1,
 'reinforcement learning': 1,
 'visual shape color': 1,
 'high calorie': 1,
 'small incentive': 1,
 'scrambled drawings': 1,
 'time conditioning': 1,
 'movement': 1,
 'reversal learning task': 1,
 'high vs low calorie food': 1,
 'Sub-Id': 1,
 'adult': 1,
 'perceptual matching task': 1,
 'inhibition': 1,
 'visual spatial cue': 1,
 'visual color pies with probabilities': 1,
 'verbal reward': 1,
 'modelling prediction error': 1,
 'feedback reward': 1,
 'precue': 1,
 'experimental task': 1,
 'conditioning task': 1,
 'monetary and erotic reward': 1,
 'two and four choice reversal learning task': 1,
 'high incentive': 1,
 'positive expectancy': 1,
 'erotic reward': 1,
 'cups task': 1,
 'safe choice': 1,
 'Stroop task': 1,
 'visual identical shapes':

### Check contains/not contains

In [517]:
df_checks = df_final.copy()

In [518]:
############
check_set = {'high reward > low reward'}
mask = df_proc.check_column_elem(df=df_checks, colummn_name='contrast_set', check_set=check_set)
with pd.option_context('display.max_colwidth', None):  # to show all the column values
    print(tabulate(df_checks[mask], headers='keys'))

    title                                                                                                                                                            author                           #Foci    #Subjects  contrast_set                                                                                                                                                                                               keywords_split
--  ---------------------------------------------------------------------------------------------------------------------------------------------------------------  -----------------------------  -------  -----------  ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  -----------------------------------------------------------------------------------------------------------------------------------------------

***

## Split wm and rest

In [519]:
df_data = df_main.copy()
mask_wm = df_data['keywords'].str.contains('working memory')

### Working Memory

In [520]:
df_wm = df_data[mask_wm]

In [521]:
df_wm['title'].unique().size

17

#### Focis number wm - for author,title and contrast

In [522]:
df_group_focis_wm = df_wm.groupby(['title', 'author', 'contrast']
                                  ).size().reset_index(name="#Foci")
df_group_focis_wm.head(1)

,title,author,contrast,#Foci
0,A Rapid fMRI Task Battery for Mappin...,Drobyshevsky et al,2-back > 0-back,13


#### Subjects number wm

In [523]:
group_subjects_wm = df_wm.groupby(['title', 'author', 'subjects', 'contrast'])
dict_keys_subjects_wm = group_subjects_wm.groups.keys()
df_group_subjects_wm = pd.DataFrame.from_records([*dict_keys_subjects_wm],
                                                 columns=['title', 'author', '#Subjects', 'contrast'])
df_group_subjects_wm.head(10)

,title,author,#Subjects,contrast
0,A Rapid fMRI Task Battery for Mappin...,Drobyshevsky et al,31,2-back > 0-back
1,A functional neuroimaging study of m...,Taylor et al,12,-
2,Abnormal neural activity in partiall...,Lee et al,14,1-back > rest
3,Alterations in the functional neural...,Cruz et al,23,-
4,An fMRI study of working memory for ...,Beneventi et al,12,2-back > 1-back > 0-back
5,Anterior cingulate cortex differentl...,Di et al,49,1-back > 2-back
6,Anterior cingulate cortex differentl...,Di et al,49,2-back > 1-back
7,Does caffeine modulate verbal workin...,Koppelstaetter et al,15,2-back > 0-back
8,Frontal Hypoactivation on Functional...,Carrion et al,13,2-back > 0-back
9,Frontal Hypoactivation on Functional...,Carrion et al,13,3-back > 0-back


In [524]:
df_final_wm = pd.merge(df_group_focis_wm, df_group_subjects_wm[['#Subjects']], left_index=True,
                       right_index=True)
df_final_wm.head(10)

,title,author,contrast,#Foci,#Subjects
0,A Rapid fMRI Task Battery for Mappin...,Drobyshevsky et al,2-back > 0-back,13,31
1,A functional neuroimaging study of m...,Taylor et al,-,23,12
2,Abnormal neural activity in partiall...,Lee et al,1-back > rest,58,14
3,Alterations in the functional neural...,Cruz et al,-,27,23
4,An fMRI study of working memory for ...,Beneventi et al,2-back > 1-back > 0-back,24,12
5,Anterior cingulate cortex differentl...,Di et al,1-back > 2-back,13,49
6,Anterior cingulate cortex differentl...,Di et al,2-back > 1-back,13,49
7,Does caffeine modulate verbal workin...,Koppelstaetter et al,2-back > 0-back,16,15
8,Frontal Hypoactivation on Functional...,Carrion et al,2-back > 0-back,16,13
9,Frontal Hypoactivation on Functional...,Carrion et al,3-back > 0-back,18,13


In [525]:
df_final_wm.shape

(24, 5)

## Get set of keywords

In [526]:
# get keywords and split them
df_group_keywords_wm = df_wm.groupby(['title', 'author', 'contrast'])['keywords'].apply(lambda x: set(x)).reset_index(
    name='keywords_set')
# df_group_keywords_wm.head(10)
df_group_keywords_wm['keywords_split'] = df_group_keywords_wm['keywords_set'].apply(df_proc.split_keys)

In [527]:
# list_key_set_wm = list(df_proc.list_of_sets_to_set(df_group_keywords_wm, 'keywords_split'))
# list_key_set_wm
set_key_set_wm_stimulus = {'visual word recognition',
                           # 'reading',
                           'verbal',
                           'verbal pseudo-words',
                           'visual numbers',
                           'visual shape',
                           'visual spatial cue',
                           'visual faces',
                           'visual letters',
                           'visual object',
                           'visual shape color',
                           'facial expression',
                           'visual identical shapes',
                           'scrambled drawings'}
set_key_set_wm_modality = {'reversal learning task',
                           # 'one-back task',
                           # 'two-back task',
                           # 'three-back task',
                           'n-back task',
                           'experimental task',
                           'perceptual matching task'}

In [528]:
# keep just the needed keywords
df_group_keywords_wm['Task Modality'] = df_group_keywords_wm['keywords_split'].apply(
    lambda x: ', '.join(x.intersection(set_key_set_wm_modality)))
df_group_keywords_wm['Task Stimulus'] = df_group_keywords_wm['keywords_split'].apply(
    lambda x: ', '.join(x.intersection(set_key_set_wm_stimulus)))

In [529]:
df_group_keywords_wm.head(8)

,title,author,contrast,keywords_set,keywords_split,Task Modality,Task Stimulus
0,A Rapid fMRI Task Battery for Mappin...,Drobyshevsky et al,2-back > 0-back,{fMRI; working memory; talairach; n-...,"{fMRI, talairach, n-back task, worki...",n-back task,visual letters
1,A functional neuroimaging study of m...,Taylor et al,-,{fMRI; working memory; decision phas...,"{fMRI, monetary reward, anticipation...",n-back task,visual shape
2,Abnormal neural activity in partiall...,Lee et al,1-back > rest,{fMRI; working memory; talairach; n-...,"{fMRI, talairach, n-back task, worki...",n-back task,visual numbers
3,Alterations in the functional neural...,Cruz et al,-,{fMRI; working memory; decision maki...,"{reversal learning task, fMRI, two a...",reversal learning task,visual identical shapes
4,An fMRI study of working memory for ...,Beneventi et al,2-back > 1-back > 0-back,{fMRI; working memory; MNI; n-back t...,"{fMRI, n-back task, working memory, ...",n-back task,"scrambled drawings, facial expression"
5,Anterior cingulate cortex differentl...,Di et al,1-back > 2-back,{fMRI; working memory; MNI; n-back t...,"{fMRI, n-back task, visual letters, ...",n-back task,visual letters
6,Anterior cingulate cortex differentl...,Di et al,2-back > 1-back,{fMRI; working memory; MNI; n-back t...,"{fMRI, n-back task, visual letters, ...",n-back task,visual letters
7,Does caffeine modulate verbal workin...,Koppelstaetter et al,2-back > 0-back,{fMRI; working memory; talairach; n-...,"{fMRI, talairach, n-back task, verba...",n-back task,verbal


In [530]:
df_final_wm = pd.merge(df_final_wm, df_group_keywords_wm[['Task Modality', 'Task Stimulus']], left_index=True,
                       right_index=True)

In [531]:
df_final_wm.head(11)

,title,author,contrast,#Foci,#Subjects,Task Modality,Task Stimulus
0,A Rapid fMRI Task Battery for Mappin...,Drobyshevsky et al,2-back > 0-back,13,31,n-back task,visual letters
1,A functional neuroimaging study of m...,Taylor et al,-,23,12,n-back task,visual shape
2,Abnormal neural activity in partiall...,Lee et al,1-back > rest,58,14,n-back task,visual numbers
3,Alterations in the functional neural...,Cruz et al,-,27,23,reversal learning task,visual identical shapes
4,An fMRI study of working memory for ...,Beneventi et al,2-back > 1-back > 0-back,24,12,n-back task,"scrambled drawings, facial expression"
5,Anterior cingulate cortex differentl...,Di et al,1-back > 2-back,13,49,n-back task,visual letters
6,Anterior cingulate cortex differentl...,Di et al,2-back > 1-back,13,49,n-back task,visual letters
7,Does caffeine modulate verbal workin...,Koppelstaetter et al,2-back > 0-back,16,15,n-back task,verbal
8,Frontal Hypoactivation on Functional...,Carrion et al,2-back > 0-back,16,13,n-back task,visual numbers
9,Frontal Hypoactivation on Functional...,Carrion et al,3-back > 0-back,18,13,n-back task,visual numbers


#### Print latex to text file - wm

In [532]:
df_final_print_wm = df_final_wm.drop(['title'], axis=1)
df_final_print_wm = df_final_print_wm.sort_values(by=['author'])
df_final_print_wm

,author,contrast,#Foci,#Subjects,Task Modality,Task Stimulus
4,Beneventi et al,2-back > 1-back > 0-back,24,12,n-back task,"scrambled drawings, facial expression"
8,Carrion et al,2-back > 0-back,16,13,n-back task,visual numbers
9,Carrion et al,3-back > 0-back,18,13,n-back task,visual numbers
3,Cruz et al,-,27,23,reversal learning task,visual identical shapes
5,Di et al,1-back > 2-back,13,49,n-back task,visual letters
6,Di et al,2-back > 1-back,13,49,n-back task,visual letters
0,Drobyshevsky et al,2-back > 0-back,13,31,n-back task,visual letters
21,Duggirala et al,2-back > 0-back,62,50,n-back task,"visual faces, visual object, visual ..."
7,Koppelstaetter et al,2-back > 0-back,16,15,n-back task,verbal
2,Lee et al,1-back > rest,58,14,n-back task,visual numbers


In [533]:
pd.set_option('display.max_colwidth', None)  # to show all the column values (-1)
latex_filepath = r'../data_db/latex/wm_reports.txt'
with open(latex_filepath, 'w') as f:
    f.write(df_final_print_wm.to_latex(index=False))
pd.set_option('display.max_colwidth', 40)

C:\Users\Chh\AppData\Local\Temp\ipykernel_31044\2813524007.py:4: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(df_final_print_wm.to_latex(index=False))


### Reward magnitude and type

In [534]:
df_reward = df_data[~mask_wm]

In [535]:
df_reward['title'].unique().size

46

In [536]:
df_reward.shape

(1330, 26)

#### Get set of keywords

In [537]:
# get keywords and split them
df_reward['keywords_split'] = df_reward['keywords'].apply(lambda x: df_proc.split_keys([x]))
df_reward.head(2)

C:\Users\Chh\AppData\Local\Temp\ipykernel_31044\2419239918.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reward['keywords_split'] = df_reward['keywords'].apply(lambda x: df_proc.split_keys([x]))


,importance,author,title,table_name,contrast,keywords,Name,Left/Right,Broadman Area,X(R),...,NV,index_col,MNIX,MNIY,MNIZ,TALX,TALY,TALZ,significance,keywords_split
0,++,Glascher et al,Determining a Role for Ventromedial ...,Table 1-1 Coordinates and significan...,ar; sr > neutral activity,fMRI; decision making; outcome phase...,mofc,l,11,-6,...,NaN,0,-6.0,24.0,-21.0,-6.436,22.720,-13.440,0.000025,"{fMRI, monetary reward, fractal imag..."
1,++,Glascher et al,Determining a Role for Ventromedial ...,Table 1-1 Coordinates and significan...,ar; sr > neutral activity,fMRI; decision making; outcome phase...,vmpfc,r,11,6,...,NaN,1,6.0,30.0,-9.0,4.724,27.408,-2.105,0.000050,"{fMRI, monetary reward, fractal imag..."


#### Create new columns

In [538]:
set_reward_type = {
    'erotic reward': 1,
    'verbal reward': 1,
    'monetary and erotic reward': 1,
    'feedback reward': 1,
    'taste reward': 2,
    'context-dependent reward': 3,  # maybe get it out
    'social reward': 7,
    'monetary reward': 36}

set_reward_contingency = {
    # 'viewing',  ## should be where is passive
    'passive',
    'gambling',
    'performance-dependent',
    'reversal learning',
    'n-back task'  # check this
}

set_reward_phase = {
    'anticipation phase',
    'decision phase',
    'outcome phase',
}

set_reward_decision_making = {
    'decision making'
}

In [539]:
set_reward_influence = {
    'negative reward': 5,
    'positive reward': 11,
}
set_keywords_others = {
    'safe choice': 1,
    'risk choice': 5,
    'positive expectancy': 1,
    'negative expectancy': 1,

}

In [540]:
df = pd.DataFrame()
list_new_cols = ['Reward Type', 'Reward Contingency', 'Reward Phase']
list_of_keys = [set_reward_type, set_reward_contingency, set_reward_phase]
for col_name, set_el in zip(list_new_cols, list_of_keys):
    df[col_name] = df_reward['keywords_split'].apply(lambda x: ', '.join(x.intersection(set_el)))

In [541]:
df['Decision Making'] = df_reward['keywords_split'].apply(
    lambda x: 'Choice' if set_reward_decision_making.issubset(x) else 'No Choice')

In [542]:
df.set_index(df_reward.index)
df_reward_final = pd.merge(df_reward, df, left_index=True,
                           right_index=True)
df_reward_final.head(2)

,importance,author,title,table_name,contrast,keywords,Name,Left/Right,Broadman Area,X(R),...,MNIZ,TALX,TALY,TALZ,significance,keywords_split,Reward Type,Reward Contingency,Reward Phase,Decision Making
0,++,Glascher et al,Determining a Role for Ventromedial ...,Table 1-1 Coordinates and significan...,ar; sr > neutral activity,fMRI; decision making; outcome phase...,mofc,l,11,-6,...,-21.0,-6.436,22.720,-13.440,0.000025,"{fMRI, monetary reward, fractal imag...",monetary reward,reversal learning,outcome phase,Choice
1,++,Glascher et al,Determining a Role for Ventromedial ...,Table 1-1 Coordinates and significan...,ar; sr > neutral activity,fMRI; decision making; outcome phase...,vmpfc,r,11,6,...,-9.0,4.724,27.408,-2.105,0.000050,"{fMRI, monetary reward, fractal imag...",monetary reward,reversal learning,outcome phase,Choice


In [543]:
for col_name in list_new_cols:
    counter_keywords_reward_type = data_util.df_counter(df=df_reward_final, column=col_name, order=1)
    print(col_name)
    print(json.dumps(counter_keywords_reward_type, indent=2, default=str))

Reward Type
{
  ">nan_values<": 0,
  "verbal reward, monetary reward": "18",
  "monetary reward, social reward": "19",
  "feedback reward": "20",
  "erotic reward": "20",
  "monetary and erotic reward": "20",
  "monetary reward, context-dependent reward": "34",
  "taste reward": "51",
  "": "70",
  "social reward": "155",
  "monetary reward": "923"
}
Reward Contingency
{
  ">nan_values<": 0,
  "reversal learning": "32",
  "n-back task": "46",
  "passive": "138",
  "performance-dependent": "311",
  "gambling": "374",
  "": "429"
}
Reward Phase
{
  ">nan_values<": 0,
  "anticipation phase, outcome phase": "21",
  "decision phase": "45",
  "": "126",
  "anticipation phase": "330",
  "outcome phase": "808"
}


In [544]:
counter_keywords_reward_type = data_util.df_counter(df=df_reward_final, column='Decision Making', order=1)
counter_keywords_reward_type

{'>nan_values<': 0, 'No Choice': 529, 'Choice': 801}

In [545]:
df_reward_final
df = df_reward_final.groupby(['title', 'author', "Reward Contingency"]).size().reset_index(name="#Foci")
df

,title,author,Reward Contingency,#Foci
0,A functional neuroimaging study of m...,Taylor et al,n-back task,18
1,A region of mesial prefrontal cortex...,Knutson et al,performance-dependent,21
2,Activity in human reward-sensitive b...,Nieuwenhuis et al,gambling,10
3,Anticipation of Increasing Monetary ...,Knutson et al,,24
4,Anticipation of monetary and social ...,Spreckelmeyer et al,,64
5,Anticipation of reward in a nonavers...,Kirscg et al,,56
6,Are Attractive People Rewarding - Se...,Cloutier et al,passive,19
7,Beautiful Faces Have Variable Reward...,Aharon et al,passive,51
8,Binge-Eating Disorder Reward Sensit...,Schienle et al,passive,12
9,Brain systems for assessing facial a...,Winston et al,passive,15
